# Packages

In [1]:
import numpy              as np
import pandas             as pd
import matplotlib.pyplot  as plt
import seaborn            as sns
import scipy.interpolate  as s
import arviz
import time
import os
from mpl_toolkits.mplot3d import Axes3D

# Paths and dataset

In [2]:
results_path = '../../LargeFilesResults/Model'  # where the outputs of this notebook will be saved

In [3]:
my_summary = pd.read_csv(os.path.join(results_path, 'allprobabilities_summ_rmshared_revised.csv'), 
                         index_col="Unnamed: 0")
posteriors = pd.read_csv(os.path.join(results_path, 'posteriors_rmshared_revised.csv'))

# Posteriors

In [ ]:
whan_classes = ['Not classified', 'Retired/Passive', 'wAGN', 'sAGN', 'SF']
palette      = np.array(['#8c510a','#bf812d','#80cdc1','#35978f','#01665e'])
figsize      = (15, 15)
alpha        = 0.6

In [ ]:
dimensions = posteriors.columns.values.reshape(5,5)
rows       = dimensions[:,0].size
columns    = dimensions[0,:].size

In [ ]:
plt.subplots(figsize=figsize)
position = 1
for each_beta in range(rows):
    for each_class in range(columns):
        name_temp = posteriors.columns.values.reshape(rows, columns)[each_beta,each_class]
        plot_temp = plt.subplot(rows, columns, position)
        sns.kdeplot(posteriors['%s' % str(name_temp)], shade=True, alpha=alpha, color=palette[each_beta])
        plt.legend(loc='upper right', fontsize=15)
        plt.axvline(x=0, linestyle="--", color='black')
        plt.xlabel(r"%s" % str(name_temp), fontsize=20)
        plt.tick_params('both', labelsize='20')
        plt.xlim([-1.8,1.8])
        plt.ylim([0,3.])      
        if (each_class==0):
            plt.ylabel("$\,$ WHAN \n %s" % whan_classes[each_beta], fontsize=20)       
        elif (each_class!=0): 
            plot_temp.yaxis.set_visible(False)            
        position+=1

plt.tight_layout(w_pad=0)
plt.savefig(os.path.join(results_path, '25betas_rmshared_revised.png'), bbox_inch='tight')
plt.savefig(os.path.join(results_path, '25betas_rmshared_revised.pdf'), bbox_inch='tight')
plt.show()

# Probability x Mass

In [ ]:
header = my_summary.keys()

In [ ]:
class01, class02, class03, class04, class05 = np.split(my_summary.values, 5)

In [ ]:
class01_df = pd.DataFrame(class01)
class01_df.columns = header

In [ ]:
class02_df = pd.DataFrame(class02)
class02_df.columns = header

In [ ]:
class03_df = pd.DataFrame(class03)
class03_df.columns = header

In [ ]:
class04_df = pd.DataFrame(class04)
class04_df.columns = header

In [ ]:
class05_df = pd.DataFrame(class05)
class05_df.columns = header

In [ ]:
plt.plot(class01_df["LOG_STELLAR_MASS"], class01_df["50%"], '.')
plt.show()

In [ ]:
class01_df.describe()

In [ ]:
unique_masses = np.unique(class01_df["LOG_STELLAR_MASS"].values)

In [ ]:
for i in range(unique_masses.size):
    if i==0:
        summary_temp = pd.DataFrame(class01_df[class01_df["LOG_STELLAR_MASS"].values==unique_masses[i]].describe())
        mean_temp    = summary_temp.loc[["mean"]]
        mean_all_df  = mean_temp
    else:
        summary_temp = pd.DataFrame(class01_df[class01_df["LOG_STELLAR_MASS"].values==unique_masses[i]].describe())
        mean_temp    = summary_temp.loc[["mean"]]
        mean_all_df  = pd.concat([mean_all_df, mean_temp])
mean_all_df

In [ ]:
plt.plot(mean_all_df['LOG_STELLAR_MASS'], mean_all_df['mean'], '-')
plt.plot(mean_all_df['LOG_STELLAR_MASS'], mean_all_df['25%'], '-')
plt.plot(mean_all_df['LOG_STELLAR_MASS'], mean_all_df['75%'], '-')
plt.plot(mean_all_df['LOG_STELLAR_MASS'], mean_all_df['2.5%'], '--')
plt.plot(mean_all_df['LOG_STELLAR_MASS'], mean_all_df['97.5%'], '--')

plt.show()

In [ ]:
mean_all_df.to_csv(os.path.join(results_path, 'data_for_plotting_mass_revised.csv'))